In [1]:
!apt-get update -y
!apt-get install -y \
libglib2.0-0 \
libnss3 \
libfontconfig1 \
gconf2-common \
libgbm1 \
udev \
libu2f-udev
!apt --fix-broken install -y
!wget -P /tmp https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/119.0.6045.105/linux64/chrome-linux64.zip
!unzip /tmp/chrome-linux64.zip -d /usr/bin/
!wget -P /tmp https://edgedl.me.gvt1.com/edgedl/chrome/chrome-for-testing/119.0.6045.105/linux64/chromedriver-linux64.zip
!unzip /tmp/chromedriver-linux64.zip -d /usr/bin/
!apt install -y python3-selenium
!pip install selenium==4.7.0
!pip install webdriver_manager

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]                             
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]                           
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]                
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]                           
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]                                
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,381 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [69.9 kB]
Get:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubu

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from lxml import etree

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

In [3]:
CHROME_BINARY_LOCATION = "/usr/bin/chrome-linux64/chrome"
CHROMEDRIVER_BINARY_LOCATION = "/usr/bin/chromedriver-linux64/chromedriver"

def add_driver_options(options):
    """
    Add configurable options
    """
    chrome_options = Options()
    for opt in options:
        chrome_options.add_argument(opt)
    return chrome_options

def initialize_driver():
    """
    Initialize the web driver
    """
    driver_config = {
        "options": [
            "--headless",
            "--no-sandbox",
            "--allow-insecure-localhost",
            "--disable-dev-shm-usage",
            # "--incognito",
            "--window-size=1920x1080",
            "user-agent=Chrome/119.0.6045.105"
        ],
    }
    options = add_driver_options(driver_config["options"])
    options.binary_location = CHROME_BINARY_LOCATION
    service = Service(executable_path=CHROMEDRIVER_BINARY_LOCATION)
    driver = webdriver.Chrome(
        service=service,
        options=options)
    return driver


In [8]:
import json
import os
import re
import time
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image
from bs4 import BeautifulSoup
from lxml import etree
import requests
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta

class Crawler:
    def __init__(self):
        self.home_page = f"https://vbpl.vn/pages/vbpq-timkiem.aspx?type=0&s=0&SearchIn=Title,Title1&Keyword=LU%E1%BA%ACT%20T%E1%BB%94%20CH%E1%BB%A8C%20VI%E1%BB%86N%20KI%E1%BB%82M%20S%C3%81T%20NH%C3%82N%20D%C3%82N%202014"
        self.root = f"https://vbpl.vn"
        self.head_page = [self.home_page]


    def get_pages(self, url):

      bs = BeautifulSoup(self.driver.page_source, 'html.parser')
      elements = bs.select('ul.listLaw li div.item p.title a')
      link_page = []
      data = {}

      try:
          for feature in elements:
            q = feature.get('href')
            print(q)
            if q not in data:
              link_page.append(q)
            print(q)
      except Exception as e:
            print(e)
      print('sssss')
      print(len(self.head_page))
      print('sssss2')
      new = []
      for link in link_page:
          real_link = self.root + link
          real_link = real_link.replace('&Keyword=LUẬT TỔ CHỨC VIỆN KIỂM SÁT NHÂN DÂN 2014','')
          new.append(real_link)
      
      return new

    def get_law_content(self, page_source, page_link):
        data = {}
        bs = BeautifulSoup(page_source, 'html.parser')
        page = etree.HTML(str(bs))
    
        try:
            # Lấy tất cả <p class="MsNormal">
            paragraphs = bs.find_all('p')
            content = []
    
            for p in paragraphs:
                text = p.get_text(strip=True)
                print(text)
                if text:
                    content.append(text)
    
            # Gộp lại thành một chuỗi lớn (hoặc bạn có thể trả về list nếu cần)
            data['content'] = "\n".join(content) if content else "N/A"
        
        except Exception as e:
            print(f"❌ Error retrieving law content: {e}")
            data['content'] = "N/A"
    
        print(f"✅ Crawled content from: {page_link}")
        print(f"↪ Length: {len(data['content'])} characters")
        
        return data

    def prepare_driver(self):
        self.driver = initialize_driver()

    def run(self, num_pages):
        

        main_url = self.head_page[num_pages]

        if os.path.exists("data/index.json"):  # index
            with open("data/index.json") as f:
                index = set(json.load(f))
        else:
            index = set()
        
        
        try:
            dataset_1_page = []
            self.driver.get(main_url)
            
            soup = BeautifulSoup(self.driver.page_source, 'html.parser')
            
            print(self.driver.current_url)
            time.sleep(5)
            list_link_pages = self.get_pages(main_url)
            print(len(list_link_pages), 'pages')
            with open("listLaw.txt", "w", encoding="utf-8") as f:
                # Duyệt qua các trang và ghi dữ liệu
                for page_link in list_link_pages:
                    print(f"Crawling {page_link}")
                    self.driver.get(page_link)
                    time.sleep(5)
                    
                    # Lấy nội dung của văn bản pháp luật
                    data = self.get_law_content(self.driver.page_source, page_link)
                    
                    # Lưu nội dung vào tệp TXT
                    if 'content' in data:
                        f.write(f"Content from: {page_link}\n")
                        f.write(data['content'])
                        f.write("\n\n" + "="*50 + "\n\n") 

            with open("data/index.json", "w") as f:
                json.dump(list(index), f)
            self.driver.delete_all_cookies()
            # self.driver.switch_to.window(self.driver.window_handles[0]); self.driver.close(); self.driver.switch_to.window(self.driver.window_handles[1])
        except Exception as e:
            print(e)

In [9]:
import time
import os
if __name__ == "__main__":
    folder_name = 'data'
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    crawler = Crawler()
    crawler.prepare_driver()

    
    start_time = time.time()
    print(1)
    data = crawler.run(0)  
    print(2)
    

    end_time = time.time()
    execution_time = end_time - start_time
    print(f" execution time: {execution_time:.2f} seconds")

    
    print("✅ Data saved")


1
https://vbpl.vn/pages/vbpq-timkiem.aspx?type=0&s=0&SearchIn=Title,Title1&Keyword=LU%E1%BA%ACT%20T%E1%BB%94%20CH%E1%BB%A8C%20VI%E1%BB%86N%20KI%E1%BB%82M%20S%C3%81T%20NH%C3%82N%20D%C3%82N%202014
/TW/Pages/vbpq-toanvan.aspx?ItemID=43951&Keyword=LUẬT TỔ CHỨC VIỆN KIỂM SÁT NHÂN DÂN 2014
/TW/Pages/vbpq-toanvan.aspx?ItemID=43951&Keyword=LUẬT TỔ CHỨC VIỆN KIỂM SÁT NHÂN DÂN 2014
/TW/Pages/vbpq-toanvan.aspx?ItemID=68053&Keyword=LUẬT TỔ CHỨC VIỆN KIỂM SÁT NHÂN DÂN 2014
/TW/Pages/vbpq-toanvan.aspx?ItemID=68053&Keyword=LUẬT TỔ CHỨC VIỆN KIỂM SÁT NHÂN DÂN 2014
/TW/Pages/vbpq-toanvan.aspx?ItemID=100826&Keyword=LUẬT TỔ CHỨC VIỆN KIỂM SÁT NHÂN DÂN 2014
/TW/Pages/vbpq-toanvan.aspx?ItemID=100826&Keyword=LUẬT TỔ CHỨC VIỆN KIỂM SÁT NHÂN DÂN 2014
sssss
1
sssss2
3 pages
Crawling https://vbpl.vn/TW/Pages/vbpq-toanvan.aspx?ItemID=43951
Trung ương
Danh sách các tỉnh & thành phố trực thuộc Trung ương
LUẬT
Tổ chức Viện kiểm sát nhân dân
__________

Căn cứ Hiến pháp nước Cộng hòa xã hội chủ nghĩa Việt Nam;
Quốc 